# 03 - Monte Carlo Pricing and SDE Discretization

Estimate option prices via Monte Carlo, report confidence intervals, and compare Euler discretization bias.

In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd()
if not (ROOT / 'src').exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

import pandas as pd

from src.config import config_dict
from src.black_scholes import bs_call_price
from src.monte_carlo import mc_price_european_gbm_terminal
from src.plotting import plot_mc_ci
from src.sde import error_vs_step_count_experiment

cfg = config_dict(fast_mode=True)


In [ ]:
bs = bs_call_price(cfg['S0'], cfg['K'], cfg['R'], cfg['Q'], cfg['SIGMA'], cfg['T'])
path_grid = [5_000, 20_000, 50_000]
rows = []
for n in path_grid:
    out = mc_price_european_gbm_terminal(
        S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
        n_paths=n, option_type='call', antithetic=True, seed=cfg['SEED']
    )
    rows.append({'n_paths': n, **out, 'bs_price': bs, 'bs_in_ci': out['ci_low'] <= bs <= out['ci_high']})

mc_df = pd.DataFrame(rows)
mc_df.to_csv(ROOT / 'results' / 'tables' / 'mc_vs_bs_ci.csv', index=False)
plot_mc_ci(mc_df, bs, ROOT / 'results' / 'figures' / 'mc_ci_plot.png')
mc_df


In [ ]:
anti = mc_price_european_gbm_terminal(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
    n_paths=20_000, option_type='call', antithetic=True, seed=cfg['SEED']
)
plain = mc_price_european_gbm_terminal(
    S0=cfg['S0'], K=cfg['K'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
    n_paths=20_000, option_type='call', antithetic=False, seed=cfg['SEED']
)
print('Std error (antithetic):', anti['std_error'])
print('Std error (plain):', plain['std_error'])

bias_df = error_vs_step_count_experiment(
    S0=cfg['S0'], r=cfg['R'], q=cfg['Q'], sigma=cfg['SIGMA'], T=cfg['T'],
    n_paths=20_000, step_counts=[12, 52, 126, 252], seed=cfg['SEED']
)
bias_df.to_csv(ROOT / 'results' / 'tables' / 'euler_bias_table.csv', index=False)
bias_df
